# 1. How to return structured data from a model

## The `.with_structured_output()` method
- Takes a schema as input specifying names, types, and description of the desired output attributes.
- The method returns a model-like Runnable, which outputs objects corresponding to the given schema.

In [43]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

#### Pydantic

In [44]:
from typing import Optional
from pydantic import BaseModel, Field

class Joke(BaseModel):
  """Joke to tell user."""
  setup: str = Field(description="The setup of the joke")
  punchline: str = Field(description="The punchline to the joke")
  rating: Optional[str] = Field(
    default=None,
    description="How funny the joke is, from 1 to 10"
  )
  
structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!', rating=None)

#### TypedDict or JSON Schema

In [45]:
from typing import Optional
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
  """Joke to tell user."""

  setup: Annotated[str, ..., "The setup of the joke"]
  punchline: Annotated[str, ..., "The punchline of the joke"]
  rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why was the cat sitting on the computer?',
 'rating': 8.0,
 'punchline': 'To keep an eye on the mouse!'}

In [46]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

Key 'parameters' is not supported in schema, ignoring


{'setup': 'Why did the cat cross the road?',
 'punchline': 'Because they are feline good!'}

### Choosing between multiple schemas

#### Pydantic

In [47]:
from typing import Union


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

structured_llm.invoke("Tell me a joke about cats")

ValueError: Unknown field for Schema: anyOf

#### TypedDict

In [ ]:
from typing import Optional, Union

from typing_extensions import Annotated, TypedDict


class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


class ConversationalResponse(TypedDict):
    """Respond in a conversational manner. Be kind and helpful."""

    response: Annotated[str, ..., "A conversational response to the user's query"]


class FinalResponse(TypedDict):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

structured_llm.invoke("Tell me a joke about cats")

{'final_output': {'response': 'Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!'}}

In [ ]:
structured_llm.invoke("How are you today?")

{'final_output': {'response': "I'm doing well, thank you for asking! How can I help you today?"}}

### Streaming

In [ ]:
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
  """Joke to tell user."""

  setup: Annotated[str, ..., "The setup of the joke"]
  punchline: Annotated[str, ..., "The punchline of the joke"]
  rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)

for chunk in structured_llm.stream("Tell me a joke about cats"):
  print(chunk)

{'setup': 'Why did the cat join the orchestra?', 'rating': 8.0, 'punchline': "Because they like to play with their 'mewsic'!"}


### Few-shot prompting

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])
few_shot_structured_llm = prompt | structured_llm

In [ ]:
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

{'setup': 'Timber',
 'rating': 7.0,
 'punchline': "Timber! Better open up, or I'll peck my way in!"}

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
  HumanMessage("Tell me a joke about planes", name="example_user"),
  AIMessage(
    "",
    name="example_assistant",
    tool_calls=[
      {
        "name": "joke",
        "args": {
          "setup": "Why don't planes ever get tired?",
          "punchline": "Because they have rest wings!",
          "rating": 2
        },
        "id": "1"
      }
    ]
  ),
  ToolMessage("", tool_call_id="1"),
  HumanMessage("Tell me another joke about planes", name="example_user"),
  AIMessage(
      "",
      name="example_assistant",
      tool_calls=[
          {
              "name": "joke",
              "args": {
                  "setup": "Cargo",
                  "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                  "rating": 10,
              },
              "id": "2",
          }
      ],
  ),
  ToolMessage("", tool_call_id="2"),
  HumanMessage("Now about caterpillars", name="example_user"),
  AIMessage(
      "",
      name="example_assistant",
      tool_calls=[
          {
              "name": "joke",
              "args": {
                  "setup": "Caterpillar",
                  "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                  "rating": 5,
              },
              "id": "3",
          }
      ],
  ),
  ToolMessage("", tool_call_id="3"),
]

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?").
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("placeholder", "{examples}"), ("human", "{input}")])
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({
  "examples": examples,
  "input": "crocodiles"
})

{'setup': 'Croc',
 'rating': 7.0,
 'punchline': "Croc-o-dile, I'm going to snap!"}

### (Advanced) Specifying the method for structuring outputs


In [49]:
structured_llm = llm.with_structured_output(Joke, method="json_mode")

structured_llm.invoke(
  "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

Joke(setup='Why are cats so bad at poker?', punchline="Because they're always feline good!", rating=None)

### (Advanced) Raw outputs

In [50]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='', additional_kwargs={'function_call': {'name': 'Joke', 'arguments': '{"setup": "Why are cats bad at poker?", "rating": 8.0, "punchline": "Because they like to play with their mouseterpiece!"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--336dc7c4-a19a-4984-9bce-03c3747652f7-0', tool_calls=[{'name': 'Joke', 'args': {'setup': 'Why are cats bad at poker?', 'rating': 8.0, 'punchline': 'Because they like to play with their mouseterpiece!'}, 'id': '3a1b1a74-86e8-4c92-b4ae-1ddbf7867ae3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 96, 'total_tokens': 187, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 57}}),
 'parsed': Joke(setup='Why are cats bad at poker?', punchline='Because they like to play with their mouseterpiece!', rating=8),
 'parsing_error': None}

## Prompting and parsing model outputs directly

### Using `PydanicOutputParser`

In [51]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Person(BaseModel):
  """Information about a person."""

  name: str = Field(..., description="The name of a person")
  height_in_meters: float = Field(
    ...,
    description="The height of a person in meters."
  )
  
class People(BaseModel):
  """Identifying information about all people in a text."""

  people: List[Person]

# Setup a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "Answer the user query. wrap the output in `json` tags\n{format_instructions}"
    ),
    (
      "human",
      "{query}"
    )
  ]
).partial(format_instructions=parser.get_format_instructions())

In [54]:
query = "Anna is 23 years old and she is 6 ft tall"

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of a person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of a person in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items": {"$ref": "#/

In [55]:
chain = prompt | llm | parser
chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8288)])

### Custom Parsing

In [56]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatMessagePromptTemplate
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )

class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]
    
# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "
            "Make sure to wrap the answer in \`\`\`json and \`\`\` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.model_json_schema()) 

# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    
    text = message.content
    pattern = r"\`\`\`json(.*?)\`\`\`"
    matches = re.findall(pattern, text, re.DOTALL)
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")
    

In [57]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.format_prompt(query=query).to_string())

System: Answer the user query. Output your answer as JSON that  matches the given schema: \`\`\`json
{'$defs': {'Person': {'description': 'Information about a person.', 'properties': {'name': {'description': 'The name of the person', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': 'The height of the person expressed in meters.', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': 'Identifying information about all people in a text.', 'properties': {'people': {'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`. Make sure to wrap the answer in \`\`\`json and \`\`\` tags
Human: Anna is 23 years old and she is 6 feet tall


In [58]:
chain = prompt | llm | extract_json

chain.invoke({"query": query})

[{'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]

## Combining with Addtional Tools

# 2. How to use a model to call tools

## Defining tool schemas

### Python functions

In [1]:
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

### LangChain Tool

### Pydantic class

In [2]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

### TypedDict class

In [3]:
from typing_extensions import Annotated, TypedDict

class add(TypedDict):
  """Add two integers."""
  a: Annotated[int, ..., "First integer"]
  b: Annotated[int, ..., "Second integer"]

class multiply(TypedDict):
  """Multiply two integers."""

  a: Annotated[int, ..., "First integer"]
  b: Annotated[int, ..., "Second integer"]

tools = [add, multiply]


In [5]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [6]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"b": 12.0, "a": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--27641626-0a7d-4ffe-9f7f-aaf9691fd495-0', tool_calls=[{'name': 'multiply', 'args': {'b': 12.0, 'a': 3.0}, 'id': '2de847c3-8734-4a2c-b44d-330a7152625d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 104, 'output_tokens': 103, 'total_tokens': 207, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 84}})

## Tool calls

In [10]:
query = "what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'add',
  'args': {'b': 49.0, 'a': 11.0},
  'id': '9c892ad0-b1ec-49c6-9fdd-4bc473e2b10e',
  'type': 'tool_call'}]

## Parsing

In [11]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field

class add(BaseModel):
  """Add two integers"""
  a: int = Field(..., description="First integer")
  b: int = Field(..., description="Second integer")

class multiply(BaseModel):
  """Multiply two integers"""
  a: int = Field(..., description="First integer")
  b: int = Field(..., description="Second integer")

chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[add(a=11, b=49)]

# 3. How to stream runnables

## Using Stream
All `Runnable` objects implement a sync method called `stream` and an async variant called `astream`.

These methods are designed to stream the final output in chunks, yielding each chunk as soon as it is available.

### LLMs and Chat Models

In [15]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [16]:
chunks = []
for chunk in model.stream("what color is the sky?"):
  chunks.append(chunk)
  print(chunk.content, end="|", flush=True)

The sky is **most commonly blue**.

This is due to a phenomenon called **Rayleigh scattering**, where shorter wavelengths of light (like blue and violet) are scattered more efficiently by the molecules in Earth's atmosphere than longer wavelengths (like| red and yellow). Because blue light is scattered in all directions, it makes the sky appear blue to our eyes.

However, the sky can be many other colors depending on various factors:

*   **Red, orange, pink, or| purple:** During **sunrise and sunset**, when the sun's light has to travel through more of the atmosphere. Most of the blue light is scattered away, leaving the longer red, orange, and yellow wavelengths to reach our eyes.
*|   **Grey or white:** On **cloudy or overcast days**, as clouds are made of water droplets or ice crystals that scatter all wavelengths of light equally, making them appear white or grey depending on their thickness and the amount of light they block|.
*   **Dark grey or even black:** During **severe thun

In [17]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
  chunks.append(chunk)
  print(chunk.content, end="|", flush="True")

The most common color of the sky during the day is **blue**.

This is due to a phenomenon called **Rayleigh scattering**. Here's a simple explanation:

*   Sunlight is made up of all the colors of the rainbow|.
*   Earth's atmosphere is made of tiny molecules of nitrogen and oxygen.
*   Blue light, which has shorter wavelengths, is scattered more efficiently by these tiny molecules than longer wavelengths like red or yellow.
*   |When sunlight enters our atmosphere, the blue light is scattered in all directions, making the sky appear blue to our eyes.

However, the sky can also be many other colors depending on the time of day, weather, and atmospheric conditions:|

*   **Orange, Red, Pink, or Purple** at sunrise or sunset: When the sun is low on the horizon, its light has to travel through more of the atmosphere. Most of the blue light has already been scattered away|, leaving the longer wavelengths (red, orange, yellow) to reach our eyes.
*   **Gray or White** when cloudy or overcast

In [18]:
chunks[0]

AIMessageChunk(content="The most common color of the sky during the day is **blue**.\n\nThis is due to a phenomenon called **Rayleigh scattering**. Here's a simple explanation:\n\n*   Sunlight is made up of all the colors of the rainbow", additional_kwargs={}, response_metadata={'safety_ratings': []}, id='run--4ec63ec7-4941-491f-afdf-f86e8ff13033', usage_metadata={'input_tokens': 7, 'output_tokens': 900, 'total_tokens': 907, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 851}})